In [1]:
from tensorflow.keras.layers import Dense, Input, MaxPooling2D, Dropout, Flatten
from tensorflow.keras.models import Sequential
from tensorflow.keras.applications import resnet
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import tensorflow as tf
from tensorflow.keras import metrics
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
batch_size = 64
width, height, channel = 400, 400, 3
no_epochs = 30
verbosity = 2
input_shape = (width, height, channel)

In [3]:
# ResNet50 모델 사용

pretrained_model = tf.keras.applications.Xception(
    include_top=False,
    weights="imagenet",
    input_tensor=None,
    input_shape=input_shape,
    pooling=None,
    classes=1000
)

In [4]:
train_datagen = ImageDataGenerator(rescale=1./255)
train_generator = train_datagen.flow_from_directory('preprocessed_data/train',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

valid_datagen = ImageDataGenerator(rescale=1./255)
valid_generator = valid_datagen.flow_from_directory('preprocessed_data/test',
                                                    target_size=(400, 400),
                                                    batch_size=batch_size,
                                                    class_mode='binary')

pretrained_model.trainable=True
set_trainable=False

for layer in pretrained_model.layers:
    if layer.name == 'block7c_project_conv':
        set_trainable=True
    if set_trainable:
        layer.trainable=True
    else:
        layer.trainable=False


model=Sequential()
model.add(pretrained_model)
model.add(MaxPooling2D())
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=[metrics.AUC(), 'accuracy'])

Found 2603 images belonging to 2 classes.
Found 650 images belonging to 2 classes.


In [5]:
es_callback = EarlyStopping(monitor='val_auc', mode='max', patience=8,
                                              verbose=1, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples//batch_size,
                    epochs = no_epochs,
                    validation_data=valid_generator,
                    validation_steps=valid_generator.samples//batch_size,
                    callbacks= [es_callback],
                    verbose=verbosity,
                   )

Epoch 1/30
40/40 - 30s - loss: 0.3729 - auc: 0.9286 - accuracy: 0.8499 - val_loss: 0.0822 - val_auc: 0.9989 - val_accuracy: 0.9875
Epoch 2/30
40/40 - 29s - loss: 0.0729 - auc: 0.9984 - accuracy: 0.9779 - val_loss: 0.0341 - val_auc: 1.0000 - val_accuracy: 0.9984
Epoch 3/30
40/40 - 30s - loss: 0.0329 - auc: 1.0000 - accuracy: 0.9972 - val_loss: 0.0234 - val_auc: 1.0000 - val_accuracy: 0.9969
Epoch 4/30
40/40 - 29s - loss: 0.0203 - auc: 1.0000 - accuracy: 0.9988 - val_loss: 0.0141 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 5/30
40/40 - 29s - loss: 0.0139 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 0.0106 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 6/30
40/40 - 29s - loss: 0.0124 - auc: 1.0000 - accuracy: 0.9996 - val_loss: 0.0079 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 7/30
40/40 - 29s - loss: 0.0085 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 0.0065 - val_auc: 1.0000 - val_accuracy: 1.0000
Epoch 8/30
40/40 - 29s - loss: 0.0064 - auc: 1.0000 - accuracy: 1.0000 - val_loss: 

In [6]:
prediction = model.predict(valid_generator)

In [7]:
import numpy as np

print(len(prediction[np.where(prediction > 0.5)]))
print(len(prediction[np.where(prediction < 0.5)]))

319
331


In [8]:
prediction

array([[9.85538840e-01],
       [1.00000000e+00],
       [9.99910593e-01],
       [9.63297904e-01],
       [9.67744172e-01],
       [7.31476909e-03],
       [1.12728896e-02],
       [9.99951839e-01],
       [9.97072577e-01],
       [1.12728896e-02],
       [9.99999881e-01],
       [9.98132885e-01],
       [1.56501005e-03],
       [7.32939178e-03],
       [9.99999762e-01],
       [9.90183532e-01],
       [1.82829984e-02],
       [1.39400819e-02],
       [9.99914169e-01],
       [9.97922719e-01],
       [9.99922395e-01],
       [9.99981761e-01],
       [9.22312260e-01],
       [9.96952295e-01],
       [4.82073892e-03],
       [9.10080016e-01],
       [7.15225637e-01],
       [9.99969721e-01],
       [9.99999642e-01],
       [9.30801749e-01],
       [9.86551046e-01],
       [1.07921101e-03],
       [2.53320597e-02],
       [1.39400819e-02],
       [9.89375949e-01],
       [1.31977284e-02],
       [6.15181141e-02],
       [6.90348968e-02],
       [9.92068410e-01],
       [9.93407607e-01],


In [9]:
tf.__version__

'2.3.1'

In [10]:
model.evaluate(valid_generator)

11/11 [==============================] - 5s 482ms/step - loss: 0.0339 - auc: 1.0000 - accuracy: 0.9985


[0.0339091531932354, 1.0, 0.9984615445137024]